# Exploratory Data Analysis (EDA)
## Notebook 03: Business Intelligence and Data Insights

This notebook provides comprehensive exploratory analysis to understand:
- Customer behavior patterns
- Revenue drivers and churn indicators
- Market segmentation opportunities
- Campaign effectiveness insights

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("Libraries imported successfully!")

In [ ]:
# Load dataset
df = pd.read_csv('../data/raw/master_dataset.csv')
print(f"Dataset: {df.shape}")
print(f"Churn rate: {df['churned'].mean():.2%}")
print(f"Average ARPU: ${df['arpu'].mean():.2f}")

## 1. Revenue Analysis

In [ ]:
# Revenue insights
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Revenue Analysis Dashboard', fontsize=16)

# ARPU distribution
axes[0, 0].hist(df['arpu'], bins=50, alpha=0.7, color='skyblue')
axes[0, 0].axvline(df['arpu'].mean(), color='red', linestyle='--', label=f'Mean: ${df["arpu"].mean():.2f}')
axes[0, 0].set_title('ARPU Distribution')
axes[0, 0].set_xlabel('ARPU ($)')
axes[0, 0].legend()

# ARPU by plan type
plan_arpu = df.groupby('plan_type')['arpu'].mean().sort_values(ascending=False)
axes[0, 1].bar(plan_arpu.index, plan_arpu.values, color='lightgreen')
axes[0, 1].set_title('Average ARPU by Plan Type')
axes[0, 1].set_ylabel('ARPU ($)')
axes[0, 1].tick_params(axis='x', rotation=45)

# Revenue distribution by city
city_revenue = df.groupby('city').agg({'arpu': 'mean', 'customer_id': 'count'}).sort_values('arpu', ascending=False)
axes[1, 0].bar(city_revenue.index[:5], city_revenue['arpu'][:5], color='coral')
axes[1, 0].set_title('Top 5 Cities by ARPU')
axes[1, 0].set_ylabel('ARPU ($)')
axes[1, 0].tick_params(axis='x', rotation=45)

# Age vs ARPU
axes[1, 1].scatter(df['age'], df['arpu'], alpha=0.5, color='purple')
axes[1, 1].set_title('Age vs ARPU Relationship')
axes[1, 1].set_xlabel('Age')
axes[1, 1].set_ylabel('ARPU ($)')

plt.tight_layout()
plt.show()

# Revenue summary statistics
print("\n=== REVENUE INSIGHTS ===")
print(f"Total monthly revenue: ${df['arpu'].sum():,.2f}")
print(f"Revenue per plan type:")
for plan, data in df.groupby('plan_type'):
    total_rev = data['arpu'].sum()
    avg_rev = data['arpu'].mean()
    customers = len(data)
    print(f"  {plan}: ${total_rev:,.2f} total, ${avg_rev:.2f} avg, {customers:,} customers")

## 2. Churn Analysis

In [ ]:
# Churn analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Customer Churn Analysis', fontsize=16)

# Churn by satisfaction score
satisfaction_churn = df.groupby(pd.cut(df['satisfaction_score'], bins=5))['churned'].mean()
axes[0, 0].bar(range(len(satisfaction_churn)), satisfaction_churn.values * 100, color='red', alpha=0.7)
axes[0, 0].set_title('Churn Rate by Satisfaction Score')
axes[0, 0].set_ylabel('Churn Rate (%)')
axes[0, 0].set_xlabel('Satisfaction Score Range')

# Churn by tenure
tenure_churn = df.groupby(pd.cut(df['tenure_months'], bins=5))['churned'].mean()
axes[0, 1].bar(range(len(tenure_churn)), tenure_churn.values * 100, color='orange', alpha=0.7)
axes[0, 1].set_title('Churn Rate by Tenure')
axes[0, 1].set_ylabel('Churn Rate (%)')
axes[0, 1].set_xlabel('Tenure Range (months)')

# Churn by plan type
plan_churn = df.groupby('plan_type')['churned'].mean()
axes[1, 0].bar(plan_churn.index, plan_churn.values * 100, color='salmon')
axes[1, 0].set_title('Churn Rate by Plan Type')
axes[1, 0].set_ylabel('Churn Rate (%)')
axes[1, 0].tick_params(axis='x', rotation=45)

# Churn impact on revenue
churned_revenue = df[df['churned'] == 1]['arpu'].sum()
total_revenue = df['arpu'].sum()
revenue_impact = churned_revenue / total_revenue * 100

labels = ['Retained Revenue', 'Lost Revenue']
sizes = [100 - revenue_impact, revenue_impact]
colors = ['lightblue', 'red']
axes[1, 1].pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title('Revenue Impact of Churn')

plt.tight_layout()
plt.show()

print(f"\n=== CHURN INSIGHTS ===")
print(f"Monthly revenue at risk: ${churned_revenue:,.2f} ({revenue_impact:.1f}%)")
print(f"Average ARPU of churned customers: ${df[df['churned'] == 1]['arpu'].mean():.2f}")
print(f"Average ARPU of retained customers: ${df[df['churned'] == 0]['arpu'].mean():.2f}")

## 3. Customer Segmentation Analysis

In [ ]:
# Customer segmentation
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Customer Segmentation Analysis', fontsize=16)

# Age distribution
axes[0, 0].hist(df['age'], bins=30, alpha=0.7, color='lightgreen')
axes[0, 0].set_title('Age Distribution')
axes[0, 0].set_xlabel('Age')
axes[0, 0].set_ylabel('Frequency')

# Income distribution
axes[0, 1].hist(df['annual_income'], bins=40, alpha=0.7, color='lightcoral')
axes[0, 1].set_title('Income Distribution')
axes[0, 1].set_xlabel('Annual Income ($)')
axes[0, 1].set_ylabel('Frequency')

# Gender distribution
gender_counts = df['gender'].value_counts()
axes[1, 0].pie(gender_counts.values, labels=gender_counts.index, autopct='%1.1f%%')
axes[1, 0].set_title('Gender Distribution')

# Geographic distribution
city_counts = df['city'].value_counts().head(8)
axes[1, 1].barh(city_counts.index, city_counts.values, color='skyblue')
axes[1, 1].set_title('Top 8 Cities by Customer Count')
axes[1, 1].set_xlabel('Number of Customers')

plt.tight_layout()
plt.show()

# Demographic insights
print("\n=== DEMOGRAPHIC INSIGHTS ===")
print(f"Average age: {df['age'].mean():.1f} years")
print(f"Average income: ${df['annual_income'].mean():,.0f}")
print(f"Gender distribution: {df['gender'].value_counts().to_dict()}")
print(f"Geographic spread: {df['city'].nunique()} cities")

## 4. Usage Patterns Analysis

In [ ]:
# Usage patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Customer Usage Patterns', fontsize=16)

# Data usage by plan
plan_data_usage = df.groupby('plan_type')['monthly_data_gb'].mean()
axes[0, 0].bar(plan_data_usage.index, plan_data_usage.values, color='lightblue')
axes[0, 0].set_title('Average Data Usage by Plan')
axes[0, 0].set_ylabel('Data Usage (GB)')
axes[0, 0].tick_params(axis='x', rotation=45)

# OTT usage by age group
df['age_group'] = pd.cut(df['age'], bins=[0, 30, 45, 60, 100], labels=['18-30', '31-45', '46-60', '60+'])
age_ott = df.groupby('age_group')['ott_usage_hours'].mean()
axes[0, 1].bar(age_ott.index, age_ott.values, color='orange')
axes[0, 1].set_title('OTT Usage by Age Group')
axes[0, 1].set_ylabel('OTT Hours/Month')

# Digital engagement
df['total_sessions'] = df['monthly_web_sessions'] + df['monthly_app_sessions']
axes[1, 0].scatter(df['age'], df['total_sessions'], alpha=0.5, color='green')
axes[1, 0].set_title('Digital Engagement vs Age')
axes[1, 0].set_xlabel('Age')
axes[1, 0].set_ylabel('Total Monthly Sessions')

# Usage efficiency by income
df['data_ratio'] = df['monthly_data_gb'] / df['data_allowance_gb']
income_quartiles = pd.qcut(df['annual_income'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
usage_by_income = df.groupby(income_quartiles)['data_ratio'].mean()
axes[1, 1].bar(usage_by_income.index, usage_by_income.values, color='purple')
axes[1, 1].set_title('Data Usage Ratio by Income Quartile')
axes[1, 1].set_ylabel('Usage Ratio')

plt.tight_layout()
plt.show()

print("\n=== USAGE INSIGHTS ===")
print(f"Average data usage: {df['monthly_data_gb'].mean():.1f} GB/month")
print(f"Average voice usage: {df['monthly_minutes'].mean():.0f} minutes/month")
print(f"Average OTT usage: {df['ott_usage_hours'].mean():.1f} hours/month")
print(f"Digital engagement: {df['total_sessions'].mean():.1f} sessions/month")

## 5. Campaign Effectiveness Analysis

In [ ]:
# Campaign analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Marketing Campaign Analysis', fontsize=16)

# Campaign exposure distribution
axes[0, 0].hist(df['campaigns_exposed'], bins=20, alpha=0.7, color='lightcoral')
axes[0, 0].set_title('Campaign Exposure Distribution')
axes[0, 0].set_xlabel('Number of Campaigns Exposed')
axes[0, 0].set_ylabel('Frequency')

# Conversion rate by channel
channel_performance = df.groupby('primary_channel').agg({
    'total_conversions': 'sum',
    'total_clicks': 'sum'
}).reset_index()
channel_performance['conversion_rate'] = channel_performance['total_conversions'] / channel_performance['total_clicks']
channel_performance = channel_performance.sort_values('conversion_rate', ascending=False)

axes[0, 1].bar(channel_performance['primary_channel'][:6], 
               channel_performance['conversion_rate'][:6] * 100, color='lightgreen')
axes[0, 1].set_title('Top 6 Channels by Conversion Rate')
axes[0, 1].set_ylabel('Conversion Rate (%)')
axes[0, 1].tick_params(axis='x', rotation=45)

# Campaign responsiveness by age
age_response = df.groupby('age_group')['total_conversions'].mean()
axes[1, 0].bar(age_response.index, age_response.values, color='skyblue')
axes[1, 0].set_title('Campaign Responsiveness by Age Group')
axes[1, 0].set_ylabel('Average Conversions')

# ROI analysis (simplified)
df['campaign_roi'] = df['total_conversions'] * 50 - df['campaigns_exposed'] * 10  # Simplified ROI calculation
roi_distribution = df[df['campaign_roi'] > 0]['campaign_roi']
axes[1, 1].hist(roi_distribution, bins=30, alpha=0.7, color='gold')
axes[1, 1].set_title('Positive Campaign ROI Distribution')
axes[1, 1].set_xlabel('Campaign ROI ($)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print("\n=== CAMPAIGN INSIGHTS ===")
print(f"Average campaigns per customer: {df['campaigns_exposed'].mean():.1f}")
print(f"Overall conversion rate: {(df['total_conversions'].sum() / df['total_clicks'].sum()):.2%}")
print(f"Best performing channel: {channel_performance.iloc[0]['primary_channel']} ({channel_performance.iloc[0]['conversion_rate']:.2%})")
print(f"Customers with positive ROI: {(df['campaign_roi'] > 0).sum():,} ({(df['campaign_roi'] > 0).mean():.1%})")

## 6. Correlation and Feature Relationships

In [ ]:
# Correlation heatmap
numeric_cols = ['age', 'annual_income', 'tenure_months', 'arpu', 'satisfaction_score',
                'monthly_data_gb', 'monthly_minutes', 'ott_usage_hours', 'total_sessions',
                'num_complaints_12m', 'churned']

corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Feature Correlation Matrix', fontsize=16)
plt.tight_layout()
plt.show()

# Key correlations with business metrics
print("\n=== KEY CORRELATIONS ===")
print("Correlations with ARPU:")
arpu_corr = df[numeric_cols].corrwith(df['arpu']).abs().sort_values(ascending=False)
print(arpu_corr.head(6))

print("\nCorrelations with Churn:")
churn_corr = df[numeric_cols].corrwith(df['churned']).abs().sort_values(ascending=False)
print(churn_corr.head(6))

## 7. Business Intelligence Summary

In [ ]:
print("=" * 60)
print("BUSINESS INTELLIGENCE SUMMARY")
print("=" * 60)

# Revenue insights
total_monthly_revenue = df['arpu'].sum()
revenue_at_risk = df[df['churned'] == 1]['arpu'].sum()

print(f"\n💰 REVENUE INSIGHTS:")
print(f"Total Monthly Revenue: ${total_monthly_revenue:,.2f}")
print(f"Revenue at Risk (Churned): ${revenue_at_risk:,.2f} ({revenue_at_risk/total_monthly_revenue:.1%})")
print(f"Potential Monthly Savings from Churn Prevention: ${revenue_at_risk:,.2f}")

# Customer insights
high_value_customers = (df['arpu'] > 75).sum()
at_risk_high_value = ((df['arpu'] > 75) & (df['churned'] == 1)).sum()

print(f"\n👥 CUSTOMER INSIGHTS:")
print(f"High-Value Customers (ARPU > $75): {high_value_customers:,} ({high_value_customers/len(df):.1%})")
print(f"High-Value Customers at Risk: {at_risk_high_value:,} ({at_risk_high_value/high_value_customers:.1%})")
print(f"Average Customer Satisfaction: {df['satisfaction_score'].mean():.1f}/10")

# Opportunity insights
low_satisfaction = (df['satisfaction_score'] <= 6).sum()
campaign_responsive = (df['total_conversions'] > 0).sum()

print(f"\n🎯 GROWTH OPPORTUNITIES:")
print(f"Customers with Low Satisfaction: {low_satisfaction:,} ({low_satisfaction/len(df):.1%})")
print(f"Campaign-Responsive Customers: {campaign_responsive:,} ({campaign_responsive/len(df):.1%})")
print(f"Cross-sell Potential (Light Users): {(df['data_ratio'] < 0.5).sum():,}")

# Dentsu-aligned insights
print(f"\n🔍 DENTSU STRATEGY ALIGNMENT:")
print(f"Digital-Native Customers (Age < 35): {(df['age'] < 35).sum():,} ({(df['age'] < 35).mean():.1%})")
print(f"High Digital Engagement: {(df['total_sessions'] > 20).sum():,} customers")
print(f"Multi-Channel Exposed: {(df['campaigns_exposed'] > 3).sum():,} customers")
print(f"Privacy-First Data: 100% synthetic with hashed IDs ✓")

print("\n" + "=" * 60)
print("EDA COMPLETE - READY FOR MODELING")
print("=" * 60)